In [1]:
import planetary_computer
import shapely
import shapely.wkt
import geojson as gj
import rasterio
import numpy as np
import pyproj
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from pystac_client.client import Client
from shapely.geometry import box
from rasterio import features, warp, windows, plot
from numpy import (
    nan as np_nan,
    ndarray,
    uint16 as np_uint16,
    errstate as np_errstate,
    where as np_where,
    iinfo as np_iinfo,
)

In [2]:
BACKGROUND_COLORS = {
    'ndvi': [(0, 0, 0)],
    'evi': [
        (165, 0, 38),
        (180, 15, 38)
    ]
}

BANDWIDTH_COLORS = [
    (112, 47, 167), #1.0
    (112, 47, 167), #.95
    (112, 47, 167), #.90
    (112, 47, 167), #.85
    (90, 7, 93),    #.80
    (90, 7, 93),    #.75
    (0, 33, 94),    #.70
    (0, 33, 94),    #.65
    (1, 112, 194),  #.60
    (0, 175, 241),  #.55
    (48, 152, 100), #.50
    (110, 173, 67), #.45
    (148, 208, 75), #.40
    (249, 231, 151),#.35
    (248, 210, 82), #.30
    (234, 125, 54), #.25
    (234, 125, 54), #.20
    (193, 1, 0),    #.15 
    (193, 1, 0),    #.10
    (193, 1, 0),    #.05
    (193, 1, 0),    #0.0
]
BANDWIDTH_COLORS +=  [(124, 126, 124)] * (len(BANDWIDTH_COLORS))
BANDWIDTH_COLORS = [tuple(c/255 for c in rgb) for rgb in BANDWIDTH_COLORS[::-1]]

In [3]:
def get_items(geojson_geometry):
    #Coleta todas as imagens da geometria no planetary computer
    catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=planetary_computer.sign_inplace)
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        intersects=geojson_geometry,
        datetime="2023-01-01/2023-02-28",
        limit=1  # Ajuste conforme necessário
    )
    items = list(search.get_items())
    
    return items

def generate_geojson(wkt):
    geometry = shapely.wkt.loads(wkt)
    geojson_geometry = shapely.geometry.mapping(geometry)

    return geojson_geometry

def generate_geojson_dif(wkt):
    g1 = shapely.wkt.loads(wkt)
    g2 = gj.Feature(geometry=g1, properties={})

    return g2
    

In [4]:
def get_raster_band(href, wkt):
    shapely_geometry = shape(wkt) #Transforma uma geometria WKT (wkt) em uma geometria Shapely
    with rasterio.open(href) as ds: #bre um arquivo raster (href)
        warped_geometry = warp.transform_geom(
            "EPSG:4326", ds.crs, shapely_geometry    #Transforma a geometria para a mesma projeção do raster
        )
        out_image, _ = mask(ds, [warped_geometry], crop=True) #Aplica uma máscara ao raster com a geometria transformada

    return out_image

def get_band_info(href, area):
    shapely_geometry = shape(area)
    with rasterio.open(href) as ds:
        warped_geometry = warp.transform_geom(
            "EPSG:4326", ds.crs, shapely_geometry
        )
        _, out_transform = mask(ds, [warped_geometry], crop=True)

        return {
            "transform": out_transform, #transformação do recorte
            "crs": int(f"{ds.crs}".split(":")[1]), #epsg do raster
        }

In [5]:
def calc_ndvi(b_nir: ndarray, b_red: ndarray) -> ndarray:
    ZERO_DIVISOR_FIX = np_iinfo(np_uint16).max * 2
    """
    Calculate the Normalized Difference Vegetation Index (NDVI) for arrays of reflectance values.

    NDVI is a measure of vegetation health and density. It is calculated using the formula:
    NDVI = (NIR - RED) / (NIR + RED)

    Parameters:
    b_nir (np.ndarray): An array of reflectance values in the near-infrared band.
    b_red (np.ndarray): An array of reflectance values in the red band.

    Returns:
    np.ndarray: An array of NDVI values, which range from -1 to 1.
                - Negative values generally indicate non-vegetated surfaces (e.g., water, barren land).
                - Values around 0 suggest sparse or no vegetation.
                - Positive values closer to 1 indicate healthy, dense vegetation.
                - np.nan is being used to hide 0 values as a mask
    """
    if len(b_nir) == 0 or len(b_red) == 0:
        return []

    b_nir = b_nir.astype(float)
    b_red = b_red.astype(float)

    denominator = b_nir + b_red
    denominator[denominator == 0] = ZERO_DIVISOR_FIX

    with np_errstate(divide="ignore", invalid="ignore"):
        ndvi = np_where(denominator != 0, (b_nir - b_red) / denominator, 0)

    return apply_filters(ndvi)



def apply_filters(index: ndarray) -> ndarray:
    """
    Apply filters to a NumPy array by modifying its values based on specific conditions.

    Parameters:
    -----------
    index : ndarray
        A NumPy array containing the data to be filtered.

    Returns:
    --------
    ndarray
        The filtered NumPy array with the following transformations:
    """
    index[index > 1] = 1.0
    index[index < -1] = -1.0
    index[index == 0] = np_nan
    return index


In [8]:
def create_ndvi_map(file_name, extent, geometry, sat_bandwidth, raster_transform, sat_crs = "EPSG:32721") -> str:

        transformer = pyproj.Transformer.from_crs("EPSG:4326", sat_crs)
        longitude, latitude = geometry.exterior.xy

        x, y = [], []
        for lon, lat in zip(longitude, latitude):
            x_coord, y_coord = transformer.transform(lat, lon)
            x.append(x_coord)
            y.append(y_coord)

        plt.figure(figsize=(3, 3))
        fig, ax = plt.subplots(figsize=(3, 3))

        ndvi_colors = BANDWIDTH_COLORS
        color_map = LinearSegmentedColormap.from_list("bandwidth", ndvi_colors, N=len(ndvi_colors))

        img = plot.show(sat_bandwidth, ax=ax, extent=extent, transform=raster_transform,
                                 cmap=color_map, interpolation='nearest', vmin=-1.0, vmax=1.0)

        ax.set_xlim([extent[0], extent[2]])
        ax.set_ylim([extent[1], extent[3]])

        ax.plot(x, y, color='blue', linewidth=2)
        ax.axis('off')
        plt.savefig('C:\\Users\\renan.granusso\\images\\' + file_name, bbox_inches='tight', pad_inches=0.1, dpi=200)
        plt.close()

        return 'C:\\Users\\renan.granusso\\images\\' + file_name

In [9]:
wkt = "Polygon ((-47.33789688206950785 -21.79630489865338561, -47.33598534379766676 -21.79509346615510168, -47.33119132717937561 -21.79962924218886045, -47.32991696833147444 -21.8018266472608957, -47.3315857715846704 -21.80199567702404551, -47.33546953188302098 -21.80340425062632903, -47.33792722394683494 -21.80410853223273548, -47.33953534344536251 -21.80503817865020721, -47.34205371926383776 -21.8048973235203043, -47.33789688206950785 -21.79630489865338561))"
epsg = 4326

format_date_time = "%Y-%m-%dT%H:%M:%S"
geojson_dict = generate_geojson(wkt)
items = get_items(geojson_dict)

duplicate_dates = []

for item in items:
    file_name = item.id + '.png'
    wkt_territory = shapely.wkt.loads(wkt)
    wkt_bbox = wkt_territory.envelope.bounds #Gera um BoundBox do WKT
    
    #Expansao do BBOX
    exp_x = (wkt_bbox[2] - wkt_bbox[0]) * 0.05 # 0.05 e uma expansao chumbada
    exp_y = (wkt_bbox[3] - wkt_bbox[1]) * 0.05
    
    # Criar um retângulo expandido com base no BBox
    expanded_area = box(wkt_bbox[0] - exp_x, wkt_bbox[1] - exp_y, wkt_bbox[2] + exp_x, wkt_bbox[3] + exp_y)
    expanded_area_geojson = generate_geojson_dif(expanded_area.wkt)
    
    band_red_href = item.assets["B04"].href
    band_nir_href = item.assets["B08"].href
    
    img_path = ''
    
    try:
        print(expanded_area_geojson)
        aoi_bounds = features.bounds(expanded_area_geojson.geometry)
        
        with rasterio.open(band_red_href) as band_red_src, rasterio.open(band_nir_href) as band_nir_src:
            warped_aoi_bounds_red = warp.transform_bounds("epsg:4326", band_red_src.crs, *aoi_bounds)
            aoi_window = windows.from_bounds(*warped_aoi_bounds_red, transform=band_red_src.transform)
            band_red = band_red_src.read(window=aoi_window)

            warped_aoi_bounds_nir = warp.transform_bounds("epsg:4326", band_nir_src.crs, *aoi_bounds)
            aoi_window = windows.from_bounds(*warped_aoi_bounds_nir, transform=band_nir_src.transform)
            nir_transform = band_nir_src.window_transform(aoi_window)
            band_nir = band_nir_src.read(window=aoi_window)
            
            img_ndvi = calc_ndvi(band_nir, band_red)
            ndvi_mean = np.mean(img_ndvi)
            ndvi_sum = np.sum(img_ndvi)
            
            image_path = create_ndvi_map(file_name, warped_aoi_bounds_nir, wkt_territory,
                                                                   img_ndvi, nir_transform)
        
    except Exception as ex:
            traceback.print_exc()

{"geometry": {"coordinates": [[[-47.32931, -21.805535], [-47.32931, -21.794596], [-47.342661, -21.794596], [-47.342661, -21.805535], [-47.32931, -21.805535]]], "type": "Polygon"}, "properties": {}, "type": "Feature"}
{"geometry": {"coordinates": [[[-47.32931, -21.805535], [-47.32931, -21.794596], [-47.342661, -21.794596], [-47.342661, -21.805535], [-47.32931, -21.805535]]], "type": "Polygon"}, "properties": {}, "type": "Feature"}
{"geometry": {"coordinates": [[[-47.32931, -21.805535], [-47.32931, -21.794596], [-47.342661, -21.794596], [-47.342661, -21.805535], [-47.32931, -21.805535]]], "type": "Polygon"}, "properties": {}, "type": "Feature"}
{"geometry": {"coordinates": [[[-47.32931, -21.805535], [-47.32931, -21.794596], [-47.342661, -21.794596], [-47.342661, -21.805535], [-47.32931, -21.805535]]], "type": "Polygon"}, "properties": {}, "type": "Feature"}
{"geometry": {"coordinates": [[[-47.32931, -21.805535], [-47.32931, -21.794596], [-47.342661, -21.794596], [-47.342661, -21.805535],

C:\Users\renan.granusso\AppData\Local\Temp\ipykernel_12304\3444929009.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(3, 3))


<Figure size 300x300 with 0 Axes>

<Figure size 300x300 with 0 Axes>

<Figure size 300x300 with 0 Axes>

...

<Figure size 300x300 with 0 Axes>

<Figure size 300x300 with 0 Axes>

<Figure size 300x300 with 0 Axes>

<Figure size 300x300 with 0 Axes>

<Figure size 300x300 with 0 Axes>